In [ ]:
import cv2
import glob
import json
import os

import gradio as gr
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
def show(filenames):
    n = 2
    m = 8
    fig, axes = plt.subplots(n, m, figsize=(20, 5))
    for i in range(n):
        for j in range(m):
            idx = (i * m) + j
            img = cv2.imread(filenames[idx])
            img[:, :, [0, 1, 2]] = img[:, :, [2, 1, 0]]
            axes[i, j].imshow(img)
    plt.show()

In [ ]:
filenames = glob.glob("/mnt/data/bill_images/*")

In [ ]:
show(filenames)

In [ ]:
def append_to_json_file(payload):

    filename = "data.json"

    if not os.path.isfile(filename):
        print(f"Creat file {filename}")
        with open(filename, "w") as f:
            json.dump({"entries": []}, f)
    
    with open(filename, "r") as f:
        data = json.load(f)

    data["entries"].append(payload)
    print(data)
    
    with open(filename, "w") as f:
        json.dump(data, f)

In [ ]:
def create_app():
    def get_image_patch(filename, patch_size=(256, 256)):
        img = cv2.imread(filename)
        img[:, :, [0, 1, 2]] = img[:, :, [2, 1, 0]]
        h, w, c = img.shape
        ph, pw = patch_size
        ry = np.random.randint(0, h - ph)
        rx = np.random.randint(0, w - pw)
        patch = img[ry:ry + ph, rx:rx + pw]
        upper_left = rx, ry
        lower_right = rx + pw, ry + ph
        coord = upper_left, lower_right
        pred_label_visible = None
        pred_label_text = None
        img = cv2.resize(img, None, fx=0.25, fy=0.25)
        return img, patch, coord, pred_label_visible, pred_label_text
        
    with gr.Blocks(theme="adam-haile/DSTheme") as demo:
    
        idx = np.random.randint(0, len(filenames))
        filename = filenames[idx]
        img, patch, coord, pred_label_visible, pred_label_text = get_image_patch(filename)
    
        label_visible_radio = gr.Radio(value=pred_label_visible, 
                                       choices=["yes", "no", "unclear"], 
                                       label="Is the bill visible in the image?")
        
        label_text_radio = gr.Radio(value=pred_label_text, 
                                    choices=["yes", "no", "unclear"], 
                                    label="Is text visible in the image?")
        
        filename_text = gr.Text(filename, 
                                label="filename", 
                                interactive=False)
        
        coord_text = gr.Text(coord, 
                             label="coord", 
                             interactive=False)
        
        output_textbox = gr.Textbox(label="Submitted data", 
                                    interactive=False)
            
        button = gr.Button("Submit")
    
        with gr.Row():
            big_image = gr.Image(img, height=500, width=500, label="Full image for reference")
            image = gr.Image(patch, height=750, width=750, label="Patch to label")
    
        @button.click(inputs=[label_visible_radio, 
                              label_text_radio, 
                              filename_text, 
                              coord_text], 
                      outputs=[output_textbox, 
                               label_visible_radio, 
                               label_text_radio, 
                               filename_text,
                               coord_text,
                               big_image,
                               image])
        def submit(label_visible, label_text, filename, coord):
            
            idx = np.random.randint(0, len(filenames))
            new_filename = filenames[idx]
            img, patch, new_coord, pred_label_visible, pred_label_text = get_image_patch(filename)
    
            entry = dict()
            entry["label_visible"] = label_visible
            entry["label_text"] = label_text
            entry["filename"] = filename
            entry["coord"] = coord
            
            append_to_json_file(entry)
            text = f"{label_visible=}, {label_text=}, {filename=}, {coord=}"
    
            return text, pred_label_visible, pred_label_text, new_filename, new_coord, img, patch
    
    _ = demo.launch(inline=False, inbrowser=True)

In [ ]:
create_app()